In [1]:
from sympy import *
from tools import *

x, y, z = symbols('x y z')
init_printing(use_unicode=True)
"""
    examples
"""
f = Func(1, "x0^2 + 1")
print(f.eval([("x0", 1)]))
print(f.diff("x0"))
print(f.grad())

ff = Func(3, "x0^2 + x1**3 + sin(x2) + 1")
print(ff.eval([("x0", 1)]))
print(ff.diff("x0"))
print(ff.grad())


2
2*x0
2*x0
x1**3 + sin(x2) + 2
2*x0
2*x0 + 3*x1**2 + cos(x2)


In [ ]:
import sympy as sp
from tools import *

"""
    examples
"""
A = sp.Matrix([[1, 0],
               [0, 1]])
b = sp.Matrix([1, 1])
c = 5
qf = QFunc(2, A, b, c)
k = 10

for i in range(10, 100):
    print(i, k)
    print(qf := create_random_quadratic_func(i, k))
    print(qf.cond())


10 10
mi = Decimal('1.666666666666666666666666667') ma = Decimal('6')
1.66666666666667*x0**2 + 2.93731671744129*x1**2 + 4.01545200603152*x2**2 + 1.93922356360545*x3**2 + 3.30175581027413*x4**2 + 4.241302959068*x5**2 + 2.43115432966739*x6**2 + 4.98512359562204*x7**2 + 2.42165822368666*x8**2 + 6.0*x9**2 + 5
lambdas = [1.66666666666667, 2.93731671744129, 4.01545200603152, 1.93922356360545, 3.30175581027413, 4.24130295906800, 2.43115432966739, 4.98512359562204, 2.42165822368666, 6.00000000000000]
L = 6.00000000000000 l = 1.66666666666667
3.60000000000000
11 10
mi = Decimal('1.111111111111111111111111111') ma = Decimal('9')
1.11111111111111*x0**2 + 1.99382846460741*x1**2 + 7.21093784171072*x10**2 + 8.13966257856689*x2**2 + 6.43893396620539*x3**2 + 8.36059585103911*x4**2 + 3.70002583049157*x5**2 + 1.14200893863077*x6**2 + 6.2972908659239*x7**2 + 2.50437923984668*x8**2 + 9.0*x9**2 + 5
lambdas = [1.11111111111111, 1.99382846460741, 7.21093784171072, 8.13966257856689, 6.43893396620539, 8.360595